<a href="https://colab.research.google.com/github/AbelAdissu/news_correlation_10ac_week0/blob/task-2/Copy_of_Classification_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
!gdown --folder --id '*******************************' --output /content/artifacts


/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Retrieving folder contents
Processing file 1bYA_pBCofEruSJa6DFEbQfKvMlv1h3Og merged_data.csv
Processing file 1xwBPp1P5lLGNSM9fLhLsL-jAE-CENAkM stratified_kfold.csv
Processing file 1j4TkdX7vVhrjRR6KIMZTiHaY3tsWThPz test.csv
Processing file 1yIKYHUagzygnzW-2L96COx5n2_uuieqv train.csv
Processing file 1B4vS1_Xbbmhy7_NURfPibOa2C0AfPCtD transformed_data.csv
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1bYA_pBCofEruSJa6DFEbQfKvMlv1h3Og
From (redirected): https://drive.google.com/uc?id=1bYA_pBCofEruSJa6DFEbQfKvMlv1h3Og&confirm=t&uuid=56ec077f-bf0f-40d9-98d5-9c12837c1ee3
To: /content/artifacts/merged_data.csv
100% 315M/315M [00:03<00:00, 104MB/s]
Downloadin

In [ ]:
import pandas as pd
import re
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset


In [ ]:
df = pd.read_csv('/content/artifacts/merged_data.csv')


In [ ]:
label_map = {
    "Breaking News": 0,
    "Politics": 1,
    "World News": 2,
    "Business/Finance": 3,
    "Technology": 4,
    "Science": 5,
    "Health": 6,
    "Entertainment": 7,
    "Sports": 8,
    "Environment": 9,
    "Crime": 10,
    "Education": 11,
    "Weather": 12,
    "Other": 13
}


In [ ]:
df['text'] = df['title'].fillna('') + ' ' + df['description'].fillna('') + ' ' + df['content'].fillna('')


In [ ]:
df = df.dropna(subset=['category'])


In [ ]:
df['labels'] = df['category'].map(label_map)


<ipython-input-9-ada69d6560c8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['labels'] = df['category'].map(label_map)


In [ ]:
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    return text

df['text'] = df['text'].apply(clean_text)


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(text):
    return tokenizer(text, padding='max_length', truncation=True, max_length=512)

df['input_ids'] = df['text'].apply(lambda x: tokenize_function(x)['input_ids'])
df['attention_mask'] = df['text'].apply(lambda x: tokenize_function(x)['attention_mask'])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Check for NaN values in the labels column
print(df['labels'].isnull().sum())


NameError: name 'df' is not defined

In [ ]:
# Map unmapped categories to 'Other'
df['labels'] = df['labels'].fillna(label_map['Other'])

# Verify that there are no NaN values left
print(df['labels'].isnull().sum())  # This should output 0


0


In [ ]:
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42, stratify=df['labels'])


In [ ]:
class NewsDataset(Dataset):
    def __init__(self, dataframe):
        self.input_ids = torch.tensor(dataframe['input_ids'].tolist())
        self.attention_mask = torch.tensor(dataframe['attention_mask'].tolist())
        self.labels = torch.tensor(dataframe['labels'].tolist())

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }


In [ ]:
train_dataset = NewsDataset(train_df)
val_dataset = NewsDataset(val_df)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)


In [ ]:
from transformers import AdamW
from transformers import get_scheduler


In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_map))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Move model to CPU since no GPU is available
device = torch.device("cpu")
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
model.train()
for epoch in range(num_epochs):
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")


In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch in val_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        _, predicted = torch.max(outputs.logits, dim=1)
        correct += (predicted == batch['labels']).sum().item()
        total += batch['labels'].size(0)

accuracy = correct / total
print(f"Validation Accuracy: {accuracy * 100:.2f}%")


NameError: name 'model' is not defined

In [ ]:
model.save_pretrained('/content/news_classification_model')
tokenizer.save_pretrained('/content/news_classification_model')
